In [1]:
import os

In [2]:
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification/research'

In [3]:
os.chdir("../")  # Move up to root directory
%pwd

'/Users/wilsvenleong/Downloads/learning-materials/chicken-disease-classification'

## 1. Update `params.yaml` and `config.yaml`

## 4. Update the Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    image_size: list
    learning_rate: float
    include_top: bool
    weights: str
    classes: int

## 5. Update the `ConfigurationManager` in `src/config/configuration.py`

In [5]:
from chicken_disease_classification.constants import *
from chicken_disease_classification.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_base_model_config(self) -> BaseModelConfig:
        base_model_preparation = self.config.base_model_preparation

        create_directories([base_model_preparation.root_dir])

        base_model_config = BaseModelConfig(
            root_dir=Path(base_model_preparation.root_dir),
            base_model_path=Path(base_model_preparation.base_model_path),
            updated_base_model_path=Path(
                base_model_preparation.updated_base_model_path
            ),
            image_size=self.params.IMAGE_SIZE,
            learning_rate=self.params.LEARNING_RATE,
            include_top=self.params.INCLUDE_TOP,
            weights=self.params.WEIGHTS,
            classes=self.params.CLASSES,
        )

        return base_model_config

## 6. Update the Components

In [6]:
from typing import Optional
import tensorflow as tf


class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.base_model_config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.base_model_config.image_size,
            weights=self.base_model_config.weights,
            include_top=self.base_model_config.include_top,
        )

        self.save_model(path=self.base_model_config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(
        model: tf.keras.Model,
        classes: int,
        learning_rate: float,
        freeze_all: Optional[bool] = True,
        freeze_till: Optional[int] = None,
    ):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax",
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.output,
            outputs=prediction,
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
        )

        full_model.summary()

        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.base_model_config.classes,
            learning_rate=self.base_model_config.learning_rate,
            freeze_all=True,
            freeze_till=None,
        )

        self.save_model(
            path=self.base_model_config.updated_base_model_path, model=self.full_model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

## 7. Update the Pipeline

In [7]:
try:
    config_manager = ConfigurationManager()
    base_model_config = config_manager.get_base_model_config()
    base_model = BaseModel(config=base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e

[ 2023-09-22 19:35:04,514 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: config/config.yaml
[ 2023-09-22 19:35:04,518 ] 33 common chicken-disease-logger -  INFO - Loaded YAML file successfully from: params.yaml
[ 2023-09-22 19:35:04,520 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts
[ 2023-09-22 19:35:04,521 ] 53 common chicken-disease-logger -  INFO - Created directory at: artifacts/base_model
[ 2023-09-22 19:35:04,907 ] 359 saving_utils tensorflow -  WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
[ 2023-09-22 19:35:04,971 ] 70 optimizer absl -  WARNING - At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.
[ 2023-09-22 19:35:04,973 ] 292 __init__ absl -  WARNING - There is a known slo

/Users/wilsvenleong/opt/anaconda3/envs/chicken/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
